## Подсчет профиля города

### Содержание:
* Подгрузка выборки
* Подсчет количества по годам
* [Подсчет количества по областям с fractional count](#second-bullet)
* Подсчет заявителей по областям с $ИТС \geq 1$

In [1]:
import pandas as pd
from glob import glob
from collections import defaultdict
import numpy as np

Подгрузим загруженную с patstat выборку

In [2]:
df = pd.DataFrame([])
for indx, path in enumerate((glob('resulttable-*\*'))):
    print(f'Количество заявок в архиве #{indx}:', pd.read_table(path, sep=';')['appln_id'].nunique())
    df = pd.concat([df, pd.read_table(path, sep=';')])

Количество заявок в архиве #0: 1532
Количество заявок в архиве #1: 1484
Количество заявок в архиве #2: 153


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: read_table is deprecated, use read_csv instead.
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: read_table is deprecated, use read_csv instead.
  after removing the cwd from sys.path.


In [3]:
df.head()

,appln_id,appln_filing_year,appln_auth,ipc_class_symbol,techn_field_nr,person_id,person_ctry_code,person_name,person_address
0,446438978,2015,US,A24F 47/00,1,51462997,NL,Fontem Holdings 4 B.V.,Amsterdam
1,446438978,2015,US,A24F 47/00,34,51462997,NL,Fontem Holdings 4 B.V.,Amsterdam
2,473248764,2014,US,A24F 47/00,10,51462997,NL,Fontem Holdings 4 B.V.,Amsterdam
3,473248764,2014,US,A24F 47/00,13,51462997,NL,Fontem Holdings 4 B.V.,Amsterdam
4,473248764,2014,US,A24F 47/00,34,51462997,NL,Fontem Holdings 4 B.V.,Amsterdam


**Посчитаем количество заявок города по годам**

In [4]:
pd.DataFrame(df.groupby('appln_filing_year').nunique()['appln_id']).T

appln_filing_year,2014,2015,2016,2017
appln_id,903,962,840,283


**Посчитаем топ заявителей по городу**
* Нужно оставить тех, кто входит в первые 2 квартиля

In [5]:
%%time
current_aplicant = []
for indx, person_name in enumerate(df.person_name.unique()):
    #df[df.person_name == person_name].groupby(['appln_filing_year', 'appln_id']).nunique()['appln_id'].groupby('appln_filing_year').size()
    aux_ = pd.DataFrame(df[df.person_name == person_name].groupby(['appln_filing_year', 'appln_id']).nunique()['appln_id'].groupby('appln_filing_year').size()).T
    #aux_.index = [person_name]
    #aux_ = pd.DataFrame(dict(aux_)).reset_index()
    current_aplicant.append([])
    current_aplicant[indx].append(person_name)
    for year in [2014, 2015, 2016, 2017]:
        try:
            current_aplicant[indx].append(aux_[year][0])
        except KeyError:
            current_aplicant[indx].append(0)

Wall time: 2.33 s


In [6]:
df_applicants_leaders = pd.DataFrame(current_aplicant, columns=['person_name', 2014, 2015, 2016, 2017]).assign(summa2014to16 = lambda x:x[2014]+x[2015]+x[2016])
df_applicants_leaders.columns = df_applicants_leaders.columns.tolist()[0:-1] + ['2014-16']

In [7]:
quantiles = df_applicants_leaders['2014-16'].quantile([0.25,0.5,0.75])
quantiles

0.25    1.0
0.50    1.0
0.75    3.0
Name: 2014-16, dtype: float64

In [8]:
df_applicants_leaders[df_applicants_leaders['2014-16'] >= quantiles[0.5]].sort_values('2014-16', ascending=False).shape

(393, 6)

In [9]:
df[['appln_id', 'appln_filing_year']].nunique()

appln_id             2988
appln_filing_year       4
dtype: int64

In [10]:
903 + 962 + 840 + 283

2988

**Посчитаем значения по областям с учетом fractional_count** <a class="anchor" id="second-bullet"></a>

> **структура словаря**:
> ```
>    Область
>        Год i
>            Число заявок за год i с учетом fractional_count
>            
>Пример
>    Область
>        2014
>            Число заявок за год i с учетом fractional_count
>        2015
>            Число заявок за год i с учетом fractional_count
>        2016
>            Число заявок за год i с учетом fractional_count
>        2017
>            Число заявок за год i с учетом fractional_count
>```

In [11]:
'''
    На вход:
        сырая табличка заявок
    На выход: словарь структуры выше, содержащий в себе количество заявок по областям по годам 
            с учетом fractional count
    
    dict_of_tech_fileds_per_city :dict: словарь для счёта по тех.областям
    tech_fileds_column :pd Series: колонка для хранения тех.областей
    appln_year :np array: (фактически: единичной длины) 
    tech_fileds_list :np array: лист со всеми (уникальными) тех.бластями заявки, 
                                длина листа равна количеству областей у заявки 
    fractional_count_per_application :float: число с плавающей точкой, равное доле, 
    которую нужно прибавить к каждой области по итогами анализа текущей заявки
    1/количество тех.областей = dfrac{1}{len(tech_fileds_list)}
'''

# создадим словарь структуры, описанной выше
dict_of_tech_fileds_per_city = defaultdict(lambda: defaultdict(np.float16))

# пройдёмся по всем уникальным заявкам в цикле и возьмём
#    А) Колонку с тех областями для каждой заявки
#    Б) Год каждой заявки
for appln_id in df['appln_id'].unique():
    tech_fileds_column = df[df['appln_id'] == appln_id].techn_field_nr
    appln_year = df[df['appln_id'] == appln_id].appln_filing_year.unique()
    # если внезапно было больше одного года для заявки, залогируем это выводом года. пока такого не было
    if len(appln_year) != 1:       
        print(appln_year)
    tech_fileds_list = tech_fileds_column.unique()
    fractional_count_per_application = 1 / len(tech_fileds_list)
    # добавим в словарь к каждой встретившеся в заявке области долю fractional_count по итогам анализа заявки
    for tech_filed in tech_fileds_list:
        dict_of_tech_fileds_per_city[tech_filed][appln_year[0]] += fractional_count_per_application

In [12]:
df_techs = pd.DataFrame(dict_of_tech_fileds_per_city).T.sort_index().replace(np.nan, 0)
df_techs = df_techs.assign(summa2014to16 = lambda x:x[2014]+x[2015]+x[2016])
df_techs.columns = df_techs.columns.tolist()[0:-1] + ['2014-16']
df_techs.head()

,2014,2015,2016,2017,2014-16
1,31.450000,31.066667,29.516667,8.392857,92.033333
2,202.583333,164.733333,133.450000,46.500000,500.766667
3,15.809524,11.083333,7.809524,9.833333,34.702381
4,27.333333,29.083333,23.583333,14.083333,80.000000
5,40.416667,22.250000,22.916667,12.833333,85.583333


In [13]:
df_techs

,2014,2015,2016,2017,2014-16
1,31.450000,31.066667,29.516667,8.392857,92.033333
2,202.583333,164.733333,133.450000,46.500000,500.766667
3,15.809524,11.083333,7.809524,9.833333,34.702381
4,27.333333,29.083333,23.583333,14.083333,80.000000
5,40.416667,22.250000,22.916667,12.833333,85.583333
6,208.759524,162.983333,155.926190,86.083333,527.669048
7,5.976190,6.333333,5.500000,4.000000,17.809524
8,10.333333,21.200000,13.700000,1.333333,45.233333
9,20.116667,14.833333,19.166667,7.250000,54.116667
10,47.126190,43.450000,54.226190,13.250000,144.802381


Получим заявителей по областям с $ИТС >= 1.1$
* Нужно убрать хвосты, равные _менее_ $5$ заявок за $2014-2016$ годы

In [14]:
filds_with_its_greater_or_equal1and1 = '''1
2
4
6
10
12
13
15
16
23
29
34'''
filds_with_its_greater_or_equal1and1 = filds_with_its_greater_or_equal1and1.split('\n')
print('string of fileds for sql: (', ', '.join(filds_with_its_greater_or_equal1and1), ')')

string of fileds for sql: ( 1, 2, 4, 6, 10, 12, 13, 15, 16, 23, 29, 34 )
